In [1]:
import pyodbc
from dotenv import dotenv_values
import pandas as pd
import warnings
import numpy as np
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from scipy.stats import skew, kurtosis
from fancyimpute import IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import re

warnings.filterwarnings('ignore')

# Set display options for Pandas DataFrame
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

#### Data Collection
##### Access the 2020 and 2021 data from Microsoft SQL Server
##### Connect to the database using provided credentials

In [2]:
# load environment variables from.env file into dictionary
environment_variables = dotenv_values('.env')

# get the values for the environment variables
server = environment_variables.get("DBS")
login = environment_variables.get("DBU")
password = environment_variables.get("DBP")
database = environment_variables.get("DBN")

# Create a database connection string using pyodbc
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={login};PWD={password}"

In [3]:
#Establish a connection to the database
try:
    connection = pyodbc.connect(connection_string)
    print("Connection established successfully",connection_string)
except Exception as e:
    print("Failed to establish connection:", e)

Connection established successfully DRIVER={SQL Server};SERVER=dap-projects-database.database.windows.net;DATABASE=dapDB;UID=LP1_learner;PWD=Hyp0th3s!$T3$t!ng


##### Schema for tables in the database

In [4]:
# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_SCHEMA = 'dbo'
        """
# Read data from the SQL query result into a DataFrame using the established database connection
schema_df = pd.read_sql(db_query, connection)

#  Check whether data has been retrieved successfully to confirm successful connection to database
try:
    scheschema_df = pd.read_sql(db_query, connection)
    print("Data retrieved successfully")
    print(schema_df)
except Exception as e:
    print("Failed to retrieve data:", e)

Data retrieved successfully
  TABLE_CATALOG TABLE_SCHEMA               TABLE_NAME  TABLE_TYPE
0         dapDB          dbo  LP1_startup_funding2021  BASE TABLE
1         dapDB          dbo  LP1_startup_funding2020  BASE TABLE


##### LP1_startup_funding2021 Data

In [4]:
#Define the SQL query to show specific tables in the database
query_2021 = """
        SELECT *
        FROM LP1_startup_funding2021        
        """
#Read data from the SQL query result into a dataframe
df_2021 = pd.read_sql(query_2021, connection)

# Display dataframe Info
df_2021.info()
print()

# Display the dataframe
df_2021.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1209 non-null   object 
 1   Founded        1208 non-null   float64
 2   HeadQuarter    1208 non-null   object 
 3   Sector         1209 non-null   object 
 4   What_it_does   1209 non-null   object 
 5   Founders       1205 non-null   object 
 6   Investor       1147 non-null   object 
 7   Amount         1206 non-null   object 
 8   Stage          781 non-null    object 
dtypes: float64(1), object(8)
memory usage: 85.1+ KB



,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None


##### LP1_startup_funding2020 Data

In [5]:
#Define the SQL query to show specific tables in the database
query_2020 = """
        SELECT *
        FROM LP1_startup_funding2020        
        """
#Read data from the SQL query result into a dataframe
df_2020 = pd.read_sql(query_2020, connection)

# Display dataframe Info
df_2020.info()
print()

# Display the dataframe
df_2020.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1055 non-null   object 
 1   Founded        842 non-null    float64
 2   HeadQuarter    961 non-null    object 
 3   Sector         1042 non-null   object 
 4   What_it_does   1055 non-null   object 
 5   Founders       1043 non-null   object 
 6   Investor       1017 non-null   object 
 7   Amount         801 non-null    float64
 8   Stage          591 non-null    object 
 9   column10       2 non-null      object 
dtypes: float64(2), object(8)
memory usage: 82.6+ KB



,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,None
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,None


#### 2019 Data from Onedrive

In [8]:
#CSV file path
file_path = '../Indian_Startup_Ecosystem/startup_funding2019.csv'
#Read data from the file path into a dataframe
df_2019 = pd.read_csv(file_path)

# Display dataframe Info
df_2019.info()
print()

# Display the dataframe
df_2019.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company Name   526 non-null    object
 1   Industry       526 non-null    object
 2   Round/Series   526 non-null    object
 3   Amount         526 non-null    object
 4   Location       526 non-null    object
 5   About Company  526 non-null    object
dtypes: object(6)
memory usage: 24.8+ KB



,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...


#### 2018 Data from Github repository

In [9]:
# url for the csv file
url = "https://raw.githubusercontent.com/Azubi-Africa/Career_Accelerator_LP1-Data_Analysis/main/startup_funding2018.csv"

# Local file path
local_path = '../Indian_Startup_Ecosystem/startup_funding2019.csv'

# Send a GET request to the URL
try:
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    # Save the response content to a local file
    with open(local_path, "wb") as f:
        f.write(response.content)
    print("File saved successfully")
except requests.exceptions.RequestException as e:
    print(f"Failed to download file: {e}")

# Read data from the file path into a dataframe
try:
    df_2018 = pd.read_csv(local_path)
    print("File read successfully")
except pd.errors.ParserError as e:
    print(f"Parser error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

# Display dataframe Info
df_2018.info()
print()
# Display the dataframe
df_2018.head(2)

File saved successfully
File read successfully
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company Name   526 non-null    object
 1   Industry       526 non-null    object
 2   Round/Series   526 non-null    object
 3   Amount         526 non-null    object
 4   Location       526 non-null    object
 5   About Company  526 non-null    object
dtypes: object(6)
memory usage: 24.8+ KB



,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...


#### Brief Data Standardization Considerations

In [9]:
# Print column names of all Dataframes
print("Column names of 2021 dataframe:")
print(df_2021.columns)

print("\nColumn names of 2020 dataframe:")
print(df_2020.columns)

print("\nColumn names of 2019 dataframe:")
print(df_2019.columns)

print("\nColumn names of 2018 dataframe:")
print(df_2018.columns)

Column names of 2021 dataframe:
Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage'],
      dtype='object')

Column names of 2020 dataframe:
Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage', 'column10'],
      dtype='object')

Column names of 2019 dataframe:
Index(['Company/Brand', 'Founded', 'HeadQuarter', 'Sector', 'What it does',
       'Founders', 'Investor', 'Amount($)', 'Stage'],
      dtype='object')

Column names of 2018 dataframe:
Index(['Company Name', 'Industry', 'Round/Series', 'Amount', 'Location',
       'About Company'],
      dtype='object')


In [10]:
# Create column names mapping dictionary

# Define the column mappings for each dataframe
column_mapping = {
    'Company_Brand': 'Company_Brand',
    'Founded': 'Founded',
    'HeadQuarter': 'HeadQuarter',
    'Sector': 'Sector',
    'What_it_does': 'What_it_does',
    'Founders': 'Founders',
    'Investor': 'Investor',
    'Amount': 'Amount',
    'column10': 'Column 10',
    'Stage': 'Stage',
    'Company/Brand': 'Company_Brand',
    'What it does': 'What_it_does',
    'Amount($)': 'Amount',
    'Company Name': 'Company_Brand',
    'Industry': 'Sector',
    'Round/Series': 'Stage',
    'Location': 'HeadQuarter',
    'About Company': 'What_it_does'
}

In [11]:
# Rename the columns

df_2021.rename(columns=column_mapping, inplace=True)
df_2020.rename(columns=column_mapping, inplace=True)
df_2019.rename(columns=column_mapping, inplace=True)
df_2018.rename(columns=column_mapping, inplace=True)

# Display renamed column names

print("Column names of 2021 dataframe:")
print(df_2021.columns)

print("\nColumn names of 2020 dataframe:")
print(df_2020.columns) 

print("\nColumn names of 2019 dataframe:")
print(df_2019.columns)

print("\nColumn names of 2018 dataframe:")
print(df_2018.columns)  

Column names of 2021 dataframe:
Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage'],
      dtype='object')

Column names of 2020 dataframe:
Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage', 'Column 10'],
      dtype='object')

Column names of 2019 dataframe:
Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage'],
      dtype='object')

Column names of 2018 dataframe:
Index(['Company_Brand', 'Sector', 'Stage', 'Amount', 'HeadQuarter',
       'What_it_does'],
      dtype='object')


#### Brief EDA on all dataframes

##### 2018 Dataframe

In [12]:
# Descriptive statistics on the dataframe
df_2018.describe().T

,count,unique,top,freq
Company_Brand,526,525,TheCollegeFever,2
Sector,526,405,—,30
Stage,526,21,Seed,280
Amount,526,198,—,148
HeadQuarter,526,50,"Bangalore, Karnataka, India",102
What_it_does,526,524,"TheCollegeFever is a hub for fun, fiesta and f...",2


In [13]:
# Check dataframe dimensions
print (df_2018.shape)
print(f"There are {df_2018.shape[0]} rows, and {df_2018.shape[1]} columns")

(526, 6)
There are 526 rows, and 6 columns


In [14]:
# Check dataframe info 
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  526 non-null    object
 1   Sector         526 non-null    object
 2   Stage          526 non-null    object
 3   Amount         526 non-null    object
 4   HeadQuarter    526 non-null    object
 5   What_it_does   526 non-null    object
dtypes: object(6)
memory usage: 24.8+ KB


In [15]:
# Check for total missing values in each column of the DataFrame
missing_values = df_2018.isnull().sum()
print()
print("Missing values in each column:")
print()
print(missing_values)
print()

# Check for total duplicated rows in the DataFrame
duplicate_count = df_2018.duplicated().sum()
print("Number of duplicated rows:", duplicate_count)
print()


Missing values in each column:

Company_Brand    0
Sector           0
Stage            0
Amount           0
HeadQuarter      0
What_it_does     0
dtype: int64

Number of duplicated rows: 1



In [16]:
df_2018["Amount"].unique()

array(['250000', '₹40,000,000', '₹65,000,000', '2000000', '—', '1600000',
       '₹16,000,000', '₹50,000,000', '₹100,000,000', '150000', '1100000',
       '₹500,000', '6000000', '650000', '₹35,000,000', '₹64,000,000',
       '₹20,000,000', '1000000', '5000000', '4000000', '₹30,000,000',
       '2800000', '1700000', '1300000', '₹5,000,000', '₹12,500,000',
       '₹15,000,000', '500000', '₹104,000,000', '₹45,000,000', '13400000',
       '₹25,000,000', '₹26,400,000', '₹8,000,000', '₹60,000', '9000000',
       '100000', '20000', '120000', '₹34,000,000', '₹342,000,000',
       '$143,145', '₹600,000,000', '$742,000,000', '₹1,000,000,000',
       '₹2,000,000,000', '$3,980,000', '$10,000', '₹100,000',
       '₹250,000,000', '$1,000,000,000', '$7,000,000', '$35,000,000',
       '₹550,000,000', '$28,500,000', '$2,000,000', '₹240,000,000',
       '₹120,000,000', '$2,400,000', '$30,000,000', '₹2,500,000,000',
       '$23,000,000', '$150,000', '$11,000,000', '₹44,000,000',
       '$3,240,000', '₹60

Prelimenary observations on 2018 dataset

The most frequent Stage was Seed

Most Headquarters were in Bangalore

#### 2019 Dataframe

In [17]:
# Descriptive statistics on the dataframe
df_2019.describe().T

,count,mean,std,min,25%,50%,75%,max
Founded,60.0,2014.533333,2.937003,2004.0,2013.0,2015.0,2016.25,2019.0


In [18]:
# Generate descriptive statistics for categorical columns in the DataFrame 
df_2019.describe(include="object").T

,count,unique,top,freq
Company_Brand,89,87,Kratikal,2
HeadQuarter,70,17,Bangalore,21
Sector,84,52,Edtech,7
What_it_does,89,88,Online meat shop,2
Founders,86,85,"Vivek Gupta, Abhay Hanjura",2
Investor,89,86,Undisclosed,3
Amount,89,50,Undisclosed,12
Stage,43,15,Series A,10


In [19]:
# Check dataframe dimensions
print (df_2019.shape)
print(f"There are {df_2019.shape[0]} rows, and {df_2019.shape[1]} columns")

(89, 9)
There are 89 rows, and 9 columns


In [20]:
# Check dataframe info
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  89 non-null     object 
 1   Founded        60 non-null     float64
 2   HeadQuarter    70 non-null     object 
 3   Sector         84 non-null     object 
 4   What_it_does   89 non-null     object 
 5   Founders       86 non-null     object 
 6   Investor       89 non-null     object 
 7   Amount         89 non-null     object 
 8   Stage          43 non-null     object 
dtypes: float64(1), object(8)
memory usage: 6.4+ KB


In [21]:
# Check for total missing values in each column of the DataFrame
missing_values = df_2019.isnull().sum()
print()
print("Missing values in each column:")
print()
print(missing_values)
print()

# Check for total duplicated rows in the DataFrame
duplicate_count = df_2019.duplicated().sum()
print("Number of duplicated rows:", duplicate_count)
print()


Missing values in each column:

Company_Brand     0
Founded          29
HeadQuarter      19
Sector            5
What_it_does      0
Founders          3
Investor          0
Amount            0
Stage            46
dtype: int64

Number of duplicated rows: 0



Prelimenary observations on 2019 dataset

The most frequent Stage was Series A

Most Headquarters were in Bangalore

The most frequent Sector was Edtech

#### 2020 Dataframe

In [22]:
# Generate descriptive statistics for categorical columns in the DataFrame 
df_2020.describe(include="object").T

,count,unique,top,freq
Company_Brand,1055,905,Nykaa,6
HeadQuarter,961,77,Bangalore,317
Sector,1042,302,Fintech,80
What_it_does,1055,990,Provides online learning classes,4
Founders,1043,927,Falguni Nayar,6
Investor,1017,848,Venture Catalysts,20
Stage,591,42,Series A,96
Column 10,2,2,Pre-Seed,1


In [23]:
# Descriptive statistics on the dataframe
df_2020.describe().T

,count,mean,std,min,25%,50%,75%,max
Founded,842.0,2.015363e+03,4.097909e+00,1973.0,2014.0,2016.0,2018.0,2.020000e+03
Amount,801.0,1.130430e+08,2.476635e+09,12700.0,1000000.0,3000000.0,11000000.0,7.000000e+10


In [24]:
# Check dataframe dimensions

print (df_2020.shape)
print(f"There are {df_2020.shape[0]} rows, and {df_2020.shape[1]} columns")

(1055, 10)
There are 1055 rows, and 10 columns


In [25]:
# Check the dataframe info
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1055 non-null   object 
 1   Founded        842 non-null    float64
 2   HeadQuarter    961 non-null    object 
 3   Sector         1042 non-null   object 
 4   What_it_does   1055 non-null   object 
 5   Founders       1043 non-null   object 
 6   Investor       1017 non-null   object 
 7   Amount         801 non-null    float64
 8   Stage          591 non-null    object 
 9   Column 10      2 non-null      object 
dtypes: float64(2), object(8)
memory usage: 82.6+ KB


In [26]:
# Check for total missing values in each column of the DataFrame
missing_values = df_2020.isnull().sum()
print()
print("Missing values in each column:")
print()
print(missing_values)
print()

# Check for total duplicated rows in the DataFrame
duplicate_count = df_2020.duplicated().sum()
print("Number of duplicated rows:", duplicate_count)
print()


Missing values in each column:

Company_Brand       0
Founded           213
HeadQuarter        94
Sector             13
What_it_does        0
Founders           12
Investor           38
Amount            254
Stage             464
Column 10        1053
dtype: int64

Number of duplicated rows: 3



Preliminary observations of the 2020 dataset

The most frequent Stage was Series A

Most Headquarters were in Bangalore

The most frequent Sector was Fintech

#### 2021 Dataframe

In [27]:
# Generate descriptive statistics for categorical columns in the DataFrame 
df_2021.describe(include="object").T

,count,unique,top,freq
Company_Brand,1209,1033,BharatPe,8
HeadQuarter,1208,70,Bangalore,426
Sector,1209,254,FinTech,122
What_it_does,1209,1143,BharatPe develops a QR code-based payment app ...,4
Founders,1205,1095,"Ashneer Grover, Shashvat Nakrani",7
Investor,1147,937,Inflection Point Ventures,24
Amount,1206,278,$Undisclosed,73
Stage,781,31,Seed,246


In [28]:
# Descriptive statistics on the dataframe
df_2021.describe().T

,count,mean,std,min,25%,50%,75%,max
Founded,1208.0,2016.655629,4.517364,1963.0,2015.0,2018.0,2020.0,2021.0


In [29]:
# Check the dataframe dimensions
print (df_2021.shape)
print(f"There are {df_2021.shape[0]} rows, and {df_2021.shape[1]} columns")

(1209, 9)
There are 1209 rows, and 9 columns


In [30]:
# Check the dataframe info
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1209 non-null   object 
 1   Founded        1208 non-null   float64
 2   HeadQuarter    1208 non-null   object 
 3   Sector         1209 non-null   object 
 4   What_it_does   1209 non-null   object 
 5   Founders       1205 non-null   object 
 6   Investor       1147 non-null   object 
 7   Amount         1206 non-null   object 
 8   Stage          781 non-null    object 
dtypes: float64(1), object(8)
memory usage: 85.1+ KB


In [31]:
# # Check for total missing values in each column of the DataFrame
missing_values = df_2021.isnull().sum()
print()
print("Missing values in each column:")
print()
print(missing_values)
print()

# Check for total duplicated rows in the DataFrame
duplicate_count = df_2021.duplicated().sum()
print("Number of duplicated rows:", duplicate_count)
print()


Missing values in each column:

Company_Brand      0
Founded            1
HeadQuarter        1
Sector             0
What_it_does       0
Founders           4
Investor          62
Amount             3
Stage            428
dtype: int64

Number of duplicated rows: 19



Preliminary observations of the 2021 dataset

The most frequent Stage was Series Seed

Most Headquarters were in Bangalore

Fintech was the most frequent sector in the database

#### General Observation from all datasets

The 2018 dataset has less columns than the rest of the datasets

The 2020 dataset has one extra column more than the 2019 and 2021 datasets

The discrepancies in the columns names has already been checked and resolved

#### Data Preparation and Cleaning

2018 Data Preparation and Cleaning  

In [32]:
# Preview Amount column of the dataset

df_2018["Amount"].unique()

array(['250000', '₹40,000,000', '₹65,000,000', '2000000', '—', '1600000',
       '₹16,000,000', '₹50,000,000', '₹100,000,000', '150000', '1100000',
       '₹500,000', '6000000', '650000', '₹35,000,000', '₹64,000,000',
       '₹20,000,000', '1000000', '5000000', '4000000', '₹30,000,000',
       '2800000', '1700000', '1300000', '₹5,000,000', '₹12,500,000',
       '₹15,000,000', '500000', '₹104,000,000', '₹45,000,000', '13400000',
       '₹25,000,000', '₹26,400,000', '₹8,000,000', '₹60,000', '9000000',
       '100000', '20000', '120000', '₹34,000,000', '₹342,000,000',
       '$143,145', '₹600,000,000', '$742,000,000', '₹1,000,000,000',
       '₹2,000,000,000', '$3,980,000', '$10,000', '₹100,000',
       '₹250,000,000', '$1,000,000,000', '$7,000,000', '$35,000,000',
       '₹550,000,000', '$28,500,000', '$2,000,000', '₹240,000,000',
       '₹120,000,000', '$2,400,000', '$30,000,000', '₹2,500,000,000',
       '$23,000,000', '$150,000', '$11,000,000', '₹44,000,000',
       '$3,240,000', '₹60

In [33]:
# Create a function to clean the Amount column of the 2018 DataFrame and convert Indian Rupees to US Dollars

def clean_amount_2018(Amount):
    try:
        Amount = str(Amount)
        # Remove commas
        Amount = Amount.replace(",", "")
        Amount = Amount.replace('—', "")
        # Check if the value is in Indian Rupees and convert US Dollars: Using USD = 68.4113 which was the average annual Indian Rupee to US Dollars in 2018
        if "₹" in Amount:
            Amount = Amount.replace("₹", "")
            return round(float(Amount) / 68.4113, 2)
        # Check if the value is in US Dollars
        elif "$" in Amount:
            Amount = Amount.replace("$", "")
            return round (float(Amount), 2)
        # check if no currency symbol is present, assume US Dollars
        else:
            return round(float(Amount), 2)
    except ValueError:
        # If the value is not a number, return NaN
        return np.nan
        
# Clean the Amount column of the 2018 DataFrame
df_2018["Amount"] = df_2018["Amount"].apply(clean_amount_2018)

In [34]:

df_2018.loc[:,['Amount']].head()

,Amount
0,250000.00
1,584698.73
2,950135.43
3,2000000.00
4,NaN


In [35]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  526 non-null    object 
 1   Sector         526 non-null    object 
 2   Stage          526 non-null    object 
 3   Amount         378 non-null    float64
 4   HeadQuarter    526 non-null    object 
 5   What_it_does   526 non-null    object 
dtypes: float64(1), object(5)
memory usage: 24.8+ KB


In [36]:

df_2018.loc[(178)]

Company_Brand                                       BuyForexOnline
Sector                                                      Travel
Stage            https://docs.google.com/spreadsheets/d/1x9ziNe...
Amount                                                   2000000.0
HeadQuarter                            Bangalore, Karnataka, India
What_it_does     BuyForexOnline.com is India's first completely...
Name: 178, dtype: object

In [37]:
# Convert the Stage column of the 2018 DataFrame into a string 

# Set the value of a specific cell in the "Stage" column to an empty string
df_2018.loc[178, ["Stage"]] = [""]

# Replace "Undisclosed" with an empty string in the entire "Stage" column
df_2018["Stage"] = df_2018["Stage"].astype(str).replace("Undisclosed", "")

In [38]:
#Verify that the row 178 of the "Stage" column has been cleaned up
df_2018.loc[(178)]

Company_Brand                                       BuyForexOnline
Sector                                                      Travel
Stage                                                             
Amount                                                   2000000.0
HeadQuarter                            Bangalore, Karnataka, India
What_it_does     BuyForexOnline.com is India's first completely...
Name: 178, dtype: object

In [39]:
# View the frequency of unique values in the Sector column: This column requires recategorization of the values in the Sector hence will be fixed after concatenation with the other dataset
sec_freq = df_2018["Sector"].value_counts(dropna=False)

# Display the frequencies
print(sec_freq)

Sector
—                                              30
Financial Services                             15
Education                                       8
Information Technology                          7
Health Care, Hospital                           5
                                               ..
Education, Gaming, Training                     1
Automotive, Search Engine, Service Industry     1
Wealth Management                               1
B2B, Farming, Marketplace                       1
Biotechnology, Health Care, Pharmaceutical      1
Name: count, Length: 405, dtype: int64


In [40]:
# Strip the first element of the "Sector" column and return the first element of the "Sector" column.

df_2018['Sector'] = df_2018.Sector.str.split(',').str[0]
df_2018['Sector'].head()

0         Brand Marketing
1             Agriculture
2                  Credit
3      Financial Services
4    E-Commerce Platforms
Name: Sector, dtype: object

In [41]:
# Preview HeadQuarter column
df_2018["HeadQuarter"].head()

0         Bangalore, Karnataka, India
1          Mumbai, Maharashtra, India
2             Gurgaon, Haryana, India
3         Noida, Uttar Pradesh, India
4    Hyderabad, Andhra Pradesh, India
Name: HeadQuarter, dtype: object

In [42]:
# Strip the first element of the "HeadQuarter" column and return the first element
df_2018['HeadQuarter'] = df_2018.HeadQuarter.str.split(',').str[0]
df_2018['HeadQuarter'].head()

0    Bangalore
1       Mumbai
2      Gurgaon
3        Noida
4    Hyderabad
Name: HeadQuarter, dtype: object

In [43]:
# Check for total duplicated rows in the DataFrame
duplicate_count = df_2018.duplicated().sum()
print("Number of duplicated rows:", duplicate_count)
print()

# Select rows where the count of missing values is greater than 1
df_2018[df_2018.isnull().sum(axis=1) > 1]

Number of duplicated rows: 1



,Company_Brand,Sector,Stage,Amount,HeadQuarter,What_it_does


In [44]:
# Convert the amount column to numeric data type
df_2018['Amount'] = pd.to_numeric(df_2018['Amount'], errors='coerce')

In [45]:
# Fill missing values in the "Amount" column with its median value
amt_med = df_2018['Amount'].median()
print(amt_med)

# Fill missing values in the "Amount" column with its median value

df_2018.fillna({'Amount': df_2018['Amount'].median()}, inplace=True)

1300000.0


2019 Data preparation and cleaning

In [46]:
# Preview info for the DataFrame
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  89 non-null     object 
 1   Founded        60 non-null     float64
 2   HeadQuarter    70 non-null     object 
 3   Sector         84 non-null     object 
 4   What_it_does   89 non-null     object 
 5   Founders       86 non-null     object 
 6   Investor       89 non-null     object 
 7   Amount         89 non-null     object 
 8   Stage          43 non-null     object 
dtypes: float64(1), object(8)
memory usage: 6.4+ KB


In [47]:
# check the missing values in the dataframe
df_2019.isnull().mean()

Company_Brand    0.000000
Founded          0.325843
HeadQuarter      0.213483
Sector           0.056180
What_it_does     0.000000
Founders         0.033708
Investor         0.000000
Amount           0.000000
Stage            0.516854
dtype: float64

In [48]:
# Preview the Amount column
df_2019["Amount"].unique()

array(['$6,300,000', '$150,000,000', '$28,000,000', '$30,000,000',
       '$6,000,000', 'Undisclosed', '$1,000,000', '$20,000,000',
       '$275,000,000', '$22,000,000', '$5,000,000', '$140,500',
       '$540,000,000', '$15,000,000', '$182,700', '$12,000,000',
       '$11,000,000', '$15,500,000', '$1,500,000', '$5,500,000',
       '$2,500,000', '$140,000', '$230,000,000', '$49,400,000',
       '$32,000,000', '$26,000,000', '$150,000', '$400,000', '$2,000,000',
       '$100,000,000', '$8,000,000', '$100,000', '$50,000,000',
       '$120,000,000', '$4,000,000', '$6,800,000', '$36,000,000',
       '$5,700,000', '$25,000,000', '$600,000', '$70,000,000',
       '$60,000,000', '$220,000', '$2,800,000', '$2,100,000',
       '$7,000,000', '$311,000,000', '$4,800,000', '$693,000,000',
       '$33,000,000'], dtype=object)

In [49]:
df_2019.isnull().mean()

Company_Brand    0.000000
Founded          0.325843
HeadQuarter      0.213483
Sector           0.056180
What_it_does     0.000000
Founders         0.033708
Investor         0.000000
Amount           0.000000
Stage            0.516854
dtype: float64

In [50]:
# clean the Amount column of the DataFrame

def clean_amount(Amount):
    try:
        Amount = str(Amount)
        #Remove the commas
        Amount = Amount.replace(',', '')
        Amount = Amount.replace('Undisclosed', '')
        #Replace the dollar sign nothing
        if '$' in Amount:
            Amount = Amount.replace('$', '')
            return round(float(Amount), 2)
        #If no currency symbol, assume USD
        else:
            return round(float(Amount), 2)
    except ValueError:
        #For non-numeric values, return NaN
        return np.nan
# Apply the cleaning function to the Amount column  

df_2019["Amount"] = df_2019["Amount"].apply(clean_amount)

In [51]:
# Verify if the anount column has been effected
df_2019.loc[:,['Amount']].head()

,Amount
0,6300000.0
1,150000000.0
2,28000000.0
3,30000000.0
4,6000000.0


In [52]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  89 non-null     object 
 1   Founded        60 non-null     float64
 2   HeadQuarter    70 non-null     object 
 3   Sector         84 non-null     object 
 4   What_it_does   89 non-null     object 
 5   Founders       86 non-null     object 
 6   Investor       89 non-null     object 
 7   Amount         77 non-null     float64
 8   Stage          43 non-null     object 
dtypes: float64(2), object(7)
memory usage: 6.4+ KB


In [53]:
# Strip the first element of the "Sector" column and return the first element of the "Sector" column.

df_2019['Sector'] = df_2019.Sector.str.split(',').str[0]
df_2019['Sector'].head()

0          Ecommerce
1             Edtech
2             Edtech
3    Interior design
4           AgriTech
Name: Sector, dtype: object

In [54]:
# Preview HeadQuarter column
df_2019["HeadQuarter"].head()

0          NaN
1       Mumbai
2       Mumbai
3      Chennai
4    Telangana
Name: HeadQuarter, dtype: object

In [55]:
# Strip the first element of the "HeadQuarter" column and return the first element
df_2019['HeadQuarter'] = df_2019.HeadQuarter.str.split(',').str[0]
df_2019['HeadQuarter'].head()

0          NaN
1       Mumbai
2       Mumbai
3      Chennai
4    Telangana
Name: HeadQuarter, dtype: object

In [56]:
# Check for total duplicated rows in the DataFrame
duplicate_count = df_2019.duplicated().sum()
print("Number of duplicated rows:", duplicate_count)
print()

# Select rows where the count of missing values is greater than 1
df_2019[df_2019.isnull().sum(axis=1) > 1]

Number of duplicated rows: 0



,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,6300000.0,NaN
5,FlytBase,NaN,Pune,Technology,A drone automation platform,Nitin Gupta,Undisclosed,NaN,NaN
6,Finly,NaN,Bangalore,SaaS,It builds software products that makes work si...,"Vivek AG, Veekshith C Rai","Social Capital, AngelList India, Gemba Capital...",NaN,NaN
8,Quantiphi,NaN,NaN,AI & Tech,It is an AI and big data services company prov...,Renuka Ramnath,Multiples Alternate Asset Management,20000000.0,Series A
10,Cub McPaws,2010.0,Mumbai,E-commerce & AR,A B2C brand that focusses on premium and comf...,"Abhay Bhat, Kinnar Shah",Venture Catalysts,NaN,NaN
11,Appnomic,NaN,Bangalore,SaaS,"It is a self-healing enterprise, the IT operat...",D Padmanabhan,Avataar Ventures,22000000.0,NaN
14,Open Secret,NaN,NaN,Food tech,It produces and sells top quality snacks,"Ahana Gautam, Udit Kejriwal",Matrix Partners,NaN,NaN
16,Byju's,2011.0,NaN,Edtech,Provides online learning classes,Byju Raveendran,"South Africa’s Naspers Ventures, the CPP Inves...",540000000.0,NaN
18,Witblox,2014.0,NaN,Edtech,Offers a range of robotics learning tools,Amit Modi,Mumbai Angels Network,182700.0,NaN
20,SalaryFits,2015.0,NaN,Fintech,A platform that promotes financial well-being ...,Renato Araujo,Brazilian VC Fund Confrapar,5000000.0,NaN


In [57]:
# Remove unwanted characters
df_2019['Founders'] = df_2019['Founders'].replace(['...', np.nan], np.NaN)


In [58]:
# Remove unwanted characters
df_2019['Founded'] = df_2019['Founded'].replace(['...', np.nan], np.NaN)

In [59]:
df_2019['Stage'].unique()

array([nan, 'Series C', 'Fresh funding', 'Series D', 'Pre series A',
       'Series A', 'Series G', 'Series B', 'Post series A',
       'Seed funding', 'Seed fund', 'Series E', 'Series F', 'Series B+',
       'Seed round', 'Pre-series A'], dtype=object)

2020 Data preparation and cleaning

In [60]:
#Preview the info of the dataset

df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1055 non-null   object 
 1   Founded        842 non-null    float64
 2   HeadQuarter    961 non-null    object 
 3   Sector         1042 non-null   object 
 4   What_it_does   1055 non-null   object 
 5   Founders       1043 non-null   object 
 6   Investor       1017 non-null   object 
 7   Amount         801 non-null    float64
 8   Stage          591 non-null    object 
 9   Column 10      2 non-null      object 
dtypes: float64(2), object(8)
memory usage: 82.6+ KB


In [61]:
# Check for total missing values in each column of the DataFrame
missing_values = df_2020.isnull().mean()
print()
print("Missing values in each column:")
print()
print(missing_values)
print()

# Check for total duplicated rows in the DataFrame
duplicate_count = df_2020.duplicated().sum()
print("Number of duplicated rows:", duplicate_count)
print()



Missing values in each column:

Company_Brand    0.000000
Founded          0.201896
HeadQuarter      0.089100
Sector           0.012322
What_it_does     0.000000
Founders         0.011374
Investor         0.036019
Amount           0.240758
Stage            0.439810
Column 10        0.998104
dtype: float64

Number of duplicated rows: 3



In [62]:
# Calculate the number of missing values in the column10 column in the 2020 dataframe
missing_values_count = df_2020["Column 10"].isna().sum()

# Calculate the total number of rows in the DataFrame
total_rows = len(df_2020)

# Calculate the percentage of missing values
percentage_missing = (missing_values_count / total_rows) * 100

# Display the percentage of missing values
print(f"Percentage of missing values in column10: {percentage_missing:.2f}%")


Percentage of missing values in column10: 99.81%


In [63]:
# Per the results above, where column10 of df_2020 is missing over 99% of its values, the column will be dropped.

df_2020.drop(columns=['Column 10'], axis=1, inplace=True)

In [64]:
# Strip the first element of the "Sector" column and return the first element of the "Sector" column.

df_2020['Sector'] = df_2020.Sector.str.split(',').str[0]
df_2020['Sector'].head()

0              AgriTech
1                EdTech
2    Hygiene management
3                Escrow
4              AgriTech
Name: Sector, dtype: object

In [65]:
# Strip the first element of the "HeadQuarter" column and return the first element
df_2020['HeadQuarter'] = df_2020.HeadQuarter.str.split(',').str[0]
df_2020['HeadQuarter'].head()

0      Chennai
1    Bangalore
2         Pune
3    New Delhi
4       Indore
Name: HeadQuarter, dtype: object

In [66]:
# Remove unwanted characters
df_2020['Founders'] = df_2020['Founders'].replace(['...', np.nan], np.NaN)

In [67]:
# Remove unwanted characters
df_2020['Founded'] = df_2020['Founded'].replace(['...', np.nan], np.NaN)

In [68]:
# clean the Amount column of the DataFrame

def clean_amount(Amount):
    try:
        Amount = str(Amount)
        #Remove the commas
        Amount = Amount.replace(',', '')
        #Replace the dollar sign nothing
        if '$' in Amount:
            Amount = Amount.replace('$', '')
            return round(float(Amount), 2)
        #If no currency symbol, assume USD
        else:
            return round(float(Amount), 2)
    except ValueError:
        #For non-numeric values, return NaN
        return np.nan
# Apply the cleaning function to the Amount column  

df_2020["Amount"] = df_2020["Amount"].apply(clean_amount)

In [69]:
# Preview the Amount column of the 2020 dataframe
df_2020["Amount"].head()


0    200000.0
1    100000.0
2         NaN
3    400000.0
4    340000.0
Name: Amount, dtype: float64

2021 Data preparation and cleaning

In [70]:
# Preview the info of the 2021 dataframe
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1209 non-null   object 
 1   Founded        1208 non-null   float64
 2   HeadQuarter    1208 non-null   object 
 3   Sector         1209 non-null   object 
 4   What_it_does   1209 non-null   object 
 5   Founders       1205 non-null   object 
 6   Investor       1147 non-null   object 
 7   Amount         1206 non-null   object 
 8   Stage          781 non-null    object 
dtypes: float64(1), object(8)
memory usage: 85.1+ KB


In [71]:
# Check for total missing values in each column of the DataFrame
missing_values = df_2021.isnull().mean()
print()
print("Missing values in each column:")
print()
print(missing_values)
print()

# Check for total duplicated rows in the DataFrame
duplicate_count = df_2021.duplicated().sum()
print("Number of duplicated rows:", duplicate_count)
print()


Missing values in each column:

Company_Brand    0.000000
Founded          0.000827
HeadQuarter      0.000827
Sector           0.000000
What_it_does     0.000000
Founders         0.003309
Investor         0.051282
Amount           0.002481
Stage            0.354012
dtype: float64

Number of duplicated rows: 19



In [72]:
df_2021['Amount'].unique()

array(['$1,200,000', '$120,000,000', '$30,000,000', '$51,000,000',
       '$2,000,000', '$188,000,000', '$200,000', 'Undisclosed',
       '$1,000,000', '$3,000,000', '$100,000', '$700,000', '$9,000,000',
       '$40,000,000', '$49,000,000', '$400,000', '$300,000',
       '$25,000,000', '$160,000,000', '$150,000', '$1,800,000',
       '$5,000,000', '$850,000', '$53,000,000', '$500,000', '$1,100,000',
       '$6,000,000', '$800,000', '$10,000,000', '$21,000,000',
       '$7,500,000', '$26,000,000', '$7,400,000', '$1,500,000',
       '$600,000', '$800,000,000', '$17,000,000', '$3,500,000',
       '$15,000,000', '$215,000,000', '$2,500,000', '$350,000,000',
       '$5,500,000', '$83,000,000', '$110,000,000', '$500,000,000',
       '$65,000,000', '$150,000,000,000', '$300,000,000', '$2,200,000',
       '$35,000,000', '$140,000,000', '$4,000,000', '$13,000,000', None,
       '$Undisclosed', '$2000000', '$800000', '$6000000', '$2500000',
       '$9500000', '$13000000', '$5000000', '$8000000',

In [73]:
# Replace numeric values in the "Amount" column with corresponding "Stages"  and "Investor" values

df_2021["Amount"].replace("Upsparks", 1200000, inplace=True)
df_2021["Amount"].replace("ah! Ventures", 300000, inplace=True)
df_2021["Amount"].replace("ITO Angel Network, LetsVenture", 300000, inplace=True)
df_2021["Amount"].replace("None", 6000000, inplace=True)
df_2021["Amount"].replace("JITO Angel Network, LetsVenture", 1000000 , inplace=True)
df_2021["Amount"].replace("Series C", 22000000, inplace=True)
df_2021["Amount"].replace("Seed", 5000000, inplace=True)
df_2021["Amount"].replace("Pre-series A", 1000000, inplace=True)

In [74]:
# clean the Amount column of the DataFrame

def clean_amount(Amount):
    try:
        Amount = str(Amount)
        #Remove the commas
        Amount = Amount.replace(',', '')
        Amount = Amount.replace('$Undisclosed', '')
        Amount = Amount.replace('Undisclosed', '')
        #Replace the dollar sign with nothing
        if '$' in Amount:
            Amount = Amount.replace('$', '')
            return round(float(Amount), 2)
        #If no currency symbol, assume USD
        else:
            return round(float(Amount), 2)
    except ValueError:
        #For non-numeric values, return NaN
        return np.nan
# Apply the cleaning function to the Amount column  

df_2021["Amount"] = df_2021["Amount"].apply(clean_amount)
        

In [75]:
df_2021['Amount'].head()

0      1200000.0
1    120000000.0
2     30000000.0
3     51000000.0
4      2000000.0
Name: Amount, dtype: float64

In [76]:
# Preview unique values in the Investor column

df_2021["Investor"].unique()

array(['BEENEXT, Entrepreneur First',
       'Unilazer Ventures, IIFL Asset Management',
       'GSV Ventures, Westbridge Capital', 'CDC Group, IDG Capital',
       'Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal', 'Vy Capital',
       'CIIE.CO, KIIT-TBI', 'Inflection Point Ventures',
       'Inflexor Ventures', None,
       '9Unicorns Accelerator Fund, Metaform Ventures',
       'SucSEED Indovation, IIM Calcutta Innovation Park',
       'Safe Planet Medicare', 'Impact Partners, C4D Partners',
       'Tiger Global Management, InnoVen Capital', 'Novo Tellus Capital',
       'Raintree Family Office, ADB arm',
       'Mumbai Angels, Narendra Shyamsukha', 'Paradigm, Kunal Shah',
       'Matrix Partners India, GIC', 'Chiratae Ventures, JAFCO Asia',
       'Mumbai Angels Network, Expert DOJO', 'GVFL',
       'Kotak Mahindra Bank, FMO', 'Kalaari Capital',
       'NB Ventures, IAN Fund',
       'Sequoia Capital India, Hummingbird Ventures',
       'Gaurav Munjal, Snehil Khanor', 'JITO Angel Net

In [77]:

# Create a boolean mask to identify rows with the $ symbol in the "Investor" column
mask = df_2021["Investor"].str.contains('\$', na=False)

# Use the .loc method to filter the DataFrame
filtered_df = df_2021.loc[mask]

# Display the filtered DataFrame
print(filtered_df)


           Company_Brand  Founded             HeadQuarter  \
242   Fullife Healthcare   2009.0  Pharmaceuticals\t#REF!   
256   Fullife Healthcare   2009.0  Pharmaceuticals\t#REF!   
257              MoEVing   2021.0         Gurugram\t#REF!   
545            AdmitKard   2016.0                   Noida   
1100            Sochcast   2020.0     Online Media\t#REF!   

                                                 Sector  \
242   Primary Business is Development and Manufactur...   
256   Primary Business is Development and Manufactur...   
257   MoEVing is India's only Electric Mobility focu...   
545                                              EdTech   
1100  Sochcast is an Audio experiences company that ...   

                                           What_it_does  \
242                                        Varun Khanna   
256                                        Varun Khanna   
257                         Vikash Mishra, Mragank Jain   
545   A tech solution for end to end caree

In [78]:
# Replace numeric values in the "Investor" column with corresponding "Founders" values

df_2021["Investor"].replace("$22000000", 'Morgan Stanley Private Equity Asia', inplace=True)
df_2021["Investor"].replace("$5000000", 'Anshuman Maheshwary, Dr Srihari Raju Kalidindi', inplace=True)
df_2021["Investor"].replace("$1000000", 'Vamsi Krishna, Pulkit Jain, Gaurav Munjal	#REF!', inplace=True)
df_2021["Investor"].replace("2000000", 'Ganesh Iyer' , inplace=True)

In [79]:
# Verify the changes made to the "Investor" column
df_2021.loc[242, 'Investor']

'Morgan Stanley Private Equity Asia'

In [80]:
# Preview the unique values of the Stages column of the 2021 dataframe

df_2021["Stage"].unique()

array(['Pre-series A', None, 'Series D', 'Series C', 'Seed', 'Series B',
       'Series E', 'Pre-seed', 'Series A', 'Pre-series B', 'Debt',
       '$1200000', 'Bridge', 'Seed+', 'Series F2', 'Series A+',
       'Series G', 'Series F', 'Series H', 'Series B3', 'PE', 'Series F1',
       'Pre-series A1', '$300000', 'Early seed', 'Series D1', '$6000000',
       '$1000000', 'Seies A', 'Pre-series', 'Series A2', 'Series I'],
      dtype=object)

In [81]:
# Create a boolean mask to identify rows with the $ symbol in the "Stage" column
mask = df_2021["Stage"].str.contains('\$', na=False)

# Use the .loc method to filter the DataFrame
filtered_df = df_2021.loc[mask]

# Display the filtered DataFrame
print(filtered_df)


        Company_Brand  Founded     HeadQuarter                  Sector  \
98            FanPlay   2020.0  Computer Games          Computer Games   
111           FanPlay   2020.0  Computer Games          Computer Games   
538       Little Leap   2020.0       New Delhi                  EdTech   
551             BHyve   2020.0          Mumbai         Human Resources   
674      MYRE Capital   2020.0          Mumbai  Commercial Real Estate   
677  Saarthi Pedagogy   2015.0       Ahmadabad                  EdTech   

                                          What_it_does  \
98   A real money game app specializing in trivia g...   
111  A real money game app specializing in trivia g...   
538                Soft Skills that make Smart Leaders   
551  A Future of Work Platform for diffusing Employ...   
674                Democratising Real Estate Ownership   
677  India's fastest growing Pedagogy company, serv...   

                                              Founders  \
98              

In [82]:
# Replace specific values in the 'Stage' column with NaN
values_to_drop = ['$1200000', '$300000', '$300000', '$6000000', '$1000000']
df_2021['Stage'] = df_2021['Stage'].replace(values_to_drop, pd.NA)

# Display the DataFrame after modification
df_2021["Stage"].unique()

array(['Pre-series A', None, 'Series D', 'Series C', 'Seed', 'Series B',
       'Series E', 'Pre-seed', 'Series A', 'Pre-series B', 'Debt', <NA>,
       'Bridge', 'Seed+', 'Series F2', 'Series A+', 'Series G',
       'Series F', 'Series H', 'Series B3', 'PE', 'Series F1',
       'Pre-series A1', 'Early seed', 'Series D1', 'Seies A',
       'Pre-series', 'Series A2', 'Series I'], dtype=object)

In [83]:
# Strip the first element of the "Sector" column.

df_2021['Sector'] = df_2021.Sector.str.split(',').str[0]
df_2021['Sector'].head(20)


0           AI startup
1               EdTech
2               EdTech
3       B2B E-commerce
4              FinTech
5        Home services
6           HealthTech
7           HealthTech
8         Tech Startup
9           E-commerce
10          HealthTech
11         B2B service
12          Helathcare
13    Renewable Energy
14          E-commerce
15         Electronics
16    Renewable Energy
17          IT startup
18              EdTech
19             FinTech
Name: Sector, dtype: object

In [84]:

df_2021["Sector"].unique()

array(['AI startup', 'EdTech', 'B2B E-commerce', 'FinTech',
       'Home services', 'HealthTech', 'Tech Startup', 'E-commerce',
       'B2B service', 'Helathcare', 'Renewable Energy', 'Electronics',
       'IT startup', 'Food & Beverages', 'Aeorspace', 'Deep Tech',
       'Dating', 'Gaming', 'Robotics', 'Retail', 'Food', 'Oil and Energy',
       'AgriTech', 'Telecommuncation', 'Milk startup', 'AI Chatbot', 'IT',
       'Logistics', 'Hospitality', 'Fashion', 'Marketing',
       'Transportation', 'LegalTech', 'Food delivery', 'Automotive',
       'SaaS startup', 'Fantasy sports', 'Video communication',
       'Social Media', 'Skill development', 'Rental', 'Recruitment',
       'HealthCare', 'Sports', 'Computer Games', 'Consumer Goods',
       'Information Technology', 'Apparel & Fashion',
       'Logistics & Supply Chain', 'Healthtech', 'Healthcare',
       'SportsTech', 'HRTech', 'Wine & Spirits',
       'Mechanical & Industrial Engineering', 'Spiritual',
       'Financial Services', 'I

In [85]:
# Display the count of the first element of the "Sector" column

sector_freq = df_2021["Sector"].value_counts(dropna=False)

# Display the frequencies

print(sector_freq)


Sector
FinTech                   122
EdTech                    102
Financial Services         61
E-commerce                 37
Food & Beverages           37
                         ... 
Manufacturing startup       1
Cryptocurrency              1
Vehicle repair startup      1
Advisory firm               1
Staffing & Recruiting       1
Name: count, Length: 253, dtype: int64


array(['AI startup', 'EdTech', None, 'FinTech', 'Home services',
       'HealthTech', 'Tech Startup', 'Helathcare', 'Renewable Energy',
       'Electronics', 'IT startup', 'Aeorspace', 'Deep Tech', 'Dating',
       'Gaming', 'Robotics', 'Retail', 'Food', 'Oil and Energy',
       'AgriTech', 'Telecommuncation', 'Milk startup', 'AI Chatbot', 'IT',
       'Logistics', 'Hospitality', 'Fashion', 'Marketing',
       'Transportation', 'LegalTech', 'Food delivery', 'Automotive',
       'SaaS startup', 'Fantasy sports', 'Video communication',
       'Social Media', 'Skill development', 'Rental', 'Recruitment',
       'HealthCare', 'Sports', 'Computer Games', 'Consumer Goods',
       'Information Technology', 'Healthtech'], dtype=object)

In [86]:
# Strip the first element of the "HeadQuarter" column and return the first element
df_2021['HeadQuarter'] = df_2021.HeadQuarter.str.split(',').str[0]
df_2021['HeadQuarter'].head()

0    Bangalore
1       Mumbai
2       Mumbai
3       Mumbai
4     Gurugram
Name: HeadQuarter, dtype: object

In [87]:
# Remove unwanted characters
df_2020['Founders'] = df_2020['Founders'].replace(['...', np.nan], np.NaN)

In [88]:
# Remove unwanted characters
df_2020['Founded'] = df_2020['Founded'].replace(['...', np.nan], np.NaN)

In [89]:
# Verify the frequency of unique values in the Sector column
stage_freq = df_2021["Stage"].value_counts(dropna=False)

# Display the frequencies
print(stage_freq)

Stage
None             428
Seed             246
Pre-series A     157
Series A         126
Series B          49
Pre-seed          47
Series C          45
Debt              27
Series D          21
Series E          18
Pre-series B      10
Series F           6
<NA>               6
Pre-series A1      4
Bridge             2
Pre-series         2
Series G           2
Series H           2
Series A2          1
Seies A            1
Series D1          1
Early seed         1
Series F2          1
Series F1          1
PE                 1
Series B3          1
Series A+          1
Seed+              1
Series I           1
Name: count, dtype: int64


Add funded year column to each dataframe for ease of comparison

In [90]:
# add funded year column to each dataframe for ease of comparison

df_2018['Funded_Year'] = pd.to_datetime('2018', format='%Y').year
df_2019['Funded_Year'] = pd.to_datetime('2019', format='%Y').year
df_2020['Funded_Year'] = pd.to_datetime('2020', format='%Y').year
df_2021['Funded_Year'] = pd.to_datetime('2021', format='%Y').year

In [91]:
# Merge all the dataframess by concatenating them vertically
df_con = pd.concat([df_2021, df_2020, df_2019, df_2018], ignore_index=True)
df_con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  2879 non-null   object 
 1   Founded        2110 non-null   float64
 2   HeadQuarter    2765 non-null   object 
 3   Sector         2861 non-null   object 
 4   What_it_does   2879 non-null   object 
 5   Founders       2334 non-null   object 
 6   Investor       2253 non-null   object 
 7   Amount         2470 non-null   float64
 8   Stage          1935 non-null   object 
 9   Funded_Year    2879 non-null   int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 225.1+ KB


In [92]:
# Preview the count of unique stages in the "Stages" column

unique_stg_count = df_con["Stage"].nunique()
print()
print("Number of unique stages in the 'Stage' column:")
print()
print(unique_stg_count)


Number of unique stages in the 'Stage' column:

70


In [93]:
# Preview the frequency of unique values in the Stage column
print(df_con["Stage"].value_counts())


Stage
Seed                        606
Series A                    305
Pre-series A                211
Series B                    134
Series C                    114
Seed Round                   69
Pre series A                 62
Pre-seed                     58
Series D                     50
Debt                         45
Venture - Series Unknown     37
Angel                        37
Series E                     30
Pre-series B                 14
Debt Financing               13
Pre Series A                 12
Series F                     11
Seed round                   10
Private Equity               10
Corporate Round               8
Pre-Seed                      8
Bridge                        8
Series H                      4
Pre seed round                4
Angel Round                   4
Pre-series A1                 4
Grant                         4
Post-IPO Equity               3
                              3
Secondary Market              3
Pre-series                    3
Se

In [94]:
# Define a categorization function to bin the Stage column into Ideation, Validation, Early Traction, Scalling and Exit Options based on cleaned Stage column

# Define the function to categorize the Stage column
def categorize_stage_column(df_con):
    # Convert the Stage column to string and lower case, handling NaN values
    df_con['Stage'] = df_con['Stage'].astype(str).str.lower()
    
    # Define the mapping of stages to categories
    stage_mapping = {
        'ideation': ['pre[ -]?seed'],
        'validation': ['seed', 'pre-series a', 'pre-series a1', 'pre-series b', 'angel'],
        'early_traction': ['series a', 'series a+', 'series a2', 'series b3', 'series d1'],
        'scaling': ['series b', 'series c', 'series d', 'series e'],
        'exit_option': [
            'series f', 'series f1', 'series f2', 'series g', 'series h', 'series i', 'private equity', 'mid series', 
            'venture - series unknown', 'post-ipo equity', 'post-ipo debt'
        ],
        'others': [
            'pre-series', 'bridge', 'corporate round', 'debt', 'undisclosed', 
            'secondary market', 'non-equity assistance', 'grant', 'nan', 'none', '<na>',
         
        ]
    }
    
    # Loop through the mapping and replace values in the Stage column
    for category, keywords in stage_mapping.items():
        pattern = '|'.join(keywords)
        df_con.loc[df_con['Stage'].str.contains(pattern, regex=True, na=False), 'Stage'] = category
    
    # Return the cleaned DataFrame
    return df_con

# Apply the transformation to the DataFrame
df_con = categorize_stage_column(df_con)




In [95]:
unique_stg_count = df_con["Stage"].nunique()
print()
print("Number of unique stages in the 'Stage' column:")
print()
print(unique_stg_count)


Number of unique stages in the 'Stage' column:

12


In [96]:
# View the frequency of unique values in the Stage column
print(df_con["Stage"].value_counts())


Stage
others            1030
validation         968
early_traction     388
scaling            338
exit_option         74
ideation            73
                     3
pe                   1
seies a              1
edge                 1
fresh funding        1
funding round        1
Name: count, dtype: int64


In [97]:
print(df_con["Stage"].unique())

['validation' 'others' 'scaling' 'ideation' 'early_traction' 'exit_option'
 'pe' 'seies a' 'edge' 'fresh funding' '' 'funding round']
